In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import _init_paths
import torch
import tensorwatch as tw
from models.model import create_model, load_model, save_model
from models.data_parallel import DataParallel
from logger import Logger
from datasets.dataset_factory import get_dataset
from trains.train_factory import train_factory

Using tensorboardX


In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import argparse
import os
import sys


class opts(object):
    def __init__(self):
        self.parser = argparse.ArgumentParser()
        # basic experiment setting
        self.parser.add_argument('--task', default='hoidet',
                                 help='ctdet | ddd | multi_pose | exdet')
        self.parser.add_argument('--dataset', default='hico',
                                 help='hico | vcoco | hoia')
        self.parser.add_argument('--exp_id', default='default')
        self.parser.add_argument('--test', action='store_true')
        self.parser.add_argument('--debug', type=int, default=0,
                                 help='level of visualization.'
                                      '1: only show the final detection results'
                                      '2: show the network output features'
                                      '3: use matplot to display'  # useful when lunching training with ipython notebook
                                      '4: save all visualizations to disk')
        self.parser.add_argument('--demo', default='',
                                 help='path to image/ image folders/ video. '
                                      'or "webcam"')
        self.parser.add_argument('--load_model', default='',
                                 help='path to pretrained model')
        self.parser.add_argument('--resume', action='store_true',
                                 help='resume an experiment. '
                                      'Reloaded the optimizer parameter and '
                                      'set load_model to model_last.pth '
                                      'in the exp dir if load_model is empty.')

        # system
        self.parser.add_argument('--gpus', default='0',
                                 help='-1 for CPU, use comma for multiple gpus')
        self.parser.add_argument('--num_workers', type=int, default=4,
                                 help='dataloader threads. 0 for single-thread.')
        self.parser.add_argument('--not_cuda_benchmark', action='store_true',
                                 help='disable when the input size is not fixed.')
        self.parser.add_argument('--seed', type=int, default=317,
                                 help='random seed')  # from CornerNet

        # log
        self.parser.add_argument('--print_iter', type=int, default=0,
                                 help='disable progress bar and print to screen.')
        self.parser.add_argument('--hide_data_time', action='store_true',
                                 help='not display time during training.')
        self.parser.add_argument('--save_all', action='store_true',
                                 help='save model to disk every 5 epochs.')
        self.parser.add_argument('--metric', default='loss',
                                 help='main metric to save best model')
        self.parser.add_argument('--vis_thresh', type=float, default=0.3,
                                 help='visualization threshold.')
        self.parser.add_argument('--debugger_theme', default='white',
                                 choices=['white', 'black'])

        # model
        self.parser.add_argument('--arch', default='dla_34',
                                 help='model architecture. Currently tested'
                                      'res_18 | resdcn_18 | dla_34 | hourglass')
        self.parser.add_argument('--head_conv', type=int, default=-1,
                                 help='conv layer channels for output head'
                                      '0 for no conv layer'
                                      '-1 for default setting: '
                                      '64 for resnets and 256 for dla.')
        self.parser.add_argument('--down_ratio', type=int, default=4,
                                 help='output stride. Currently only supports 4.')

        # input
        self.parser.add_argument('--input_res', type=int, default=-1,
                                 help='input height and width. -1 for default from '
                                      'dataset. Will be overriden by input_h | input_w')
        self.parser.add_argument('--input_h', type=int, default=-1,
                                 help='input height. -1 for default from dataset.')
        self.parser.add_argument('--input_w', type=int, default=-1,
                                 help='input width. -1 for default from dataset.')

        # train
        self.parser.add_argument('--lr', type=float, default=1.25e-4,
                                 help='learning rate for batch size 32.')
        self.parser.add_argument('--lr_step', type=str, default='90,120',
                                 help='drop learning rate by 10.')
        self.parser.add_argument('--num_epochs', type=int, default=140,
                                 help='total training epochs.')
        self.parser.add_argument('--batch_size', type=int, default=32,
                                 help='batch size')
        self.parser.add_argument('--master_batch_size', type=int, default=-1,
                                 help='batch size on the master gpu.')
        self.parser.add_argument('--num_iters', type=int, default=-1,
                                 help='default: #samples / batch_size.')
        self.parser.add_argument('--val_intervals', type=int, default=100000,
                                 help='number of epochs to run validation.')
        self.parser.add_argument('--trainval', action='store_true',
                                 help='include validation in training and '
                                      'test on test set')

        # test
        self.parser.add_argument('--flip_test', action='store_true',
                                 help='flip data augmentation.')
        self.parser.add_argument('--test_scales', type=str, default='1',
                                 help='multi scale test augmentation.')
        self.parser.add_argument('--nms', action='store_true',
                                 help='run nms in testing.')
        self.parser.add_argument('--K', type=int, default=100,
                                 help='max number of output objects.')
        self.parser.add_argument('--not_prefetch_test', action='store_true',
                                 help='not use parallal data pre-processing.')
        self.parser.add_argument('--fix_res', action='store_true',
                                 help='fix testing resolution or keep '
                                      'the original resolution')
        self.parser.add_argument('--keep_res', action='store_true',
                                 help='keep the original resolution'
                                      ' during validation.')
        self.parser.add_argument('--save_predictions', action='store_true',
                                 help='saving predictions when testing')
        self.parser.add_argument('--test_with_eval', action='store_true',
                                 help='do evaluation when testing')
        self.parser.add_argument('--test_video', action='store_true',
                                 help='inference with a video')
        self.parser.add_argument('--test_dir', type=str,default='',
                                 help='the video path')
        self.parser.add_argument('--save_video', type=str,default='',
                                 help='the video save path')

        # dataset
        self.parser.add_argument('--not_rand_crop', action='store_true',
                                 help='not use the random crop data augmentation'
                                      'from CornerNet.')
        self.parser.add_argument('--shift', type=float, default=0.1,
                                 help='when not using random crop'
                                      'apply shift augmentation.')
        self.parser.add_argument('--scale', type=float, default=0.4,
                                 help='when not using random crop'
                                      'apply scale augmentation.')
        self.parser.add_argument('--rotate', type=float, default=0,
                                 help='when not using random crop'
                                      'apply rotation augmentation.')
        self.parser.add_argument('--flip', type=float, default=0.5,
                                 help='probability of applying flip augmentation.')
        self.parser.add_argument('--no_color_aug', action='store_true',
                                 help='not use the color augmenation '
                                      'from CornerNet')


        # loss
        self.parser.add_argument('--mse_loss', action='store_true',
                                 help='use mse loss or focal loss to train '
                                      'keypoint heatmaps.')
        # ctdet
        self.parser.add_argument('--reg_loss', default='l1',
                                 help='regression loss: sl1 | l1 | l2')
        self.parser.add_argument('--hm_weight', type=float, default=1,
                                 help='loss weight for keypoint heatmaps.')
        self.parser.add_argument('--off_weight', type=float, default=1,
                                 help='loss weight for keypoint local offsets.')
        self.parser.add_argument('--wh_weight', type=float, default=0.1,
                                 help='loss weight for bounding box size.')

        # task
        # ctdet
        self.parser.add_argument('--norm_wh', action='store_true',
                                 help='L1(\hat(y) / y, 1) or L1(\hat(y), y)')
        self.parser.add_argument('--dense_wh', action='store_true',
                                 help='apply weighted regression near center or '
                                      'just apply regression on center point.')
        self.parser.add_argument('--cat_spec_wh', action='store_true',
                                 help='category specific bounding box size.')
        self.parser.add_argument('--not_reg_offset', action='store_true',
                                 help='not regress local offset.')

        # ground truth validation
        self.parser.add_argument('--image_dir', type=str, default='images/trainval',
                                 help='training dataset path.')
        self.parser.add_argument('--root_path', type=str, default='../Dataset',
                                 help='training dataset path.')
        self.parser.add_argument('--use_cos', type=int, default=0
                                 , help='whether using cosine lr step policy')
        self.parser.add_argument('--use_verb_sub', type=int, default=0
                                 , help='whether using verb categories for subject')

    def parse(self, args=''):
        opt = self.parser.parse_known_args()[0]
        print(opt.task)
        opt.gpus_str = opt.gpus
        opt.gpus = [int(gpu) for gpu in opt.gpus.split(',')]
        opt.gpus = [i for i in range(len(opt.gpus))] if opt.gpus[0] >= 0 else [-1]
        opt.lr_step = [int(i) for i in opt.lr_step.split(',')]
        opt.test_scales = [float(i) for i in opt.test_scales.split(',')]

        opt.fix_res = not opt.keep_res
        print('Fix size testing.' if opt.fix_res else 'Keep resolution testing.')
        opt.reg_offset = not opt.not_reg_offset

        if opt.head_conv == -1:  # init default head_conv
            opt.head_conv = 256 if 'dla' in opt.arch else 64
        opt.pad = 127 if 'hourglass' in opt.arch else 31
        opt.num_stacks = 2 if opt.arch == 'hourglass' else 1

        if opt.trainval:
            opt.val_intervals = 100000000

        if opt.debug > 0:
            opt.num_workers = 0
            opt.batch_size = 1
            opt.gpus = [opt.gpus[0]]
            opt.master_batch_size = -1

        if opt.master_batch_size == -1:
            opt.master_batch_size = opt.batch_size // len(opt.gpus)
        rest_batch_size = (opt.batch_size - opt.master_batch_size)
        opt.chunk_sizes = [opt.master_batch_size]
        for i in range(len(opt.gpus) - 1):
            slave_chunk_size = rest_batch_size // (len(opt.gpus) - 1)
            if i < rest_batch_size % (len(opt.gpus) - 1):
                slave_chunk_size += 1
            opt.chunk_sizes.append(slave_chunk_size)
        print('training chunk_sizes:', opt.chunk_sizes)


        if opt.resume and opt.load_model == '':
            model_path = opt.save_dir[:-4] if opt.save_dir.endswith('TEST') \
                else opt.save_dir
            opt.load_model = os.path.join(model_path, 'model_last.pth')
        return opt

    def update_dataset_info_and_set_heads(self, opt, dataset):
        input_h, input_w = dataset.default_resolution
        opt.mean, opt.std = dataset.mean, dataset.std
        opt.num_classes = dataset.num_classes
        opt.num_classes_verb = dataset.num_classes_verb
        # input_h(w): opt.input_h overrides opt.input_res overrides dataset default
        input_h = opt.input_res if opt.input_res > 0 else input_h
        input_w = opt.input_res if opt.input_res > 0 else input_w
        opt.input_h = opt.input_h if opt.input_h > 0 else input_h
        opt.input_w = opt.input_w if opt.input_w > 0 else input_w
        opt.output_h = opt.input_h // opt.down_ratio
        opt.output_w = opt.input_w // opt.down_ratio
        opt.input_res = max(opt.input_h, opt.input_w)
        opt.output_res = max(opt.output_h, opt.output_w)


        if opt.task == 'hoidet':
            assert opt.dataset in ['hico', 'vcoco', 'hoia']
            opt.heads = {'hm': opt.num_classes,
                         'wh': 2 if not opt.cat_spec_wh else 2 * opt.num_classes,
                         'hm_rel': opt.num_classes_verb,
                         'sub_offset': 2 * opt.num_classes_verb,
                         'obj_offset': 2 * opt.num_classes_verb}
            if opt.reg_offset:
                opt.heads.update({'reg': 2})
        else:
            assert 0, 'task not defined!'
        print('heads', opt.heads)
        return opt

    def init(self, args=''):
        default_dataset_info = {
            'hoidet': {'default_resolution': [512, 512], 'num_classes': 80,
                  'mean': [0.408, 0.447, 0.470], 'std': [0.289, 0.274, 0.278],
                  'dataset': 'hico', 'num_classes_verb': 117}
        }

        class Struct:
            def __init__(self, entries):
                for k, v in entries.items():
                    self.__setattr__(k, v)

        opt = self.parse(args)
        dataset = Struct(default_dataset_info[opt.task])
        opt.dataset = dataset.dataset
        opt = self.update_dataset_info_and_set_heads(opt, dataset)
        return opt


In [3]:
opt = opts().parse()

The history saving thread hit an unexpected error (OperationalError('database is locked')).History will not be written to the database.hoidet

Fix size testing.
training chunk_sizes: [32]


In [4]:
Dataset = get_dataset(opt.dataset)
opt = opts().update_dataset_info_and_set_heads(opt, Dataset)
print(opt)


heads {'hm': 80, 'wh': 2, 'hm_rel': 117, 'sub_offset': 234, 'obj_offset': 234, 'reg': 2}
Namespace(K=100, arch='dla_34', batch_size=32, cat_spec_wh=False, chunk_sizes=[32], dataset='hico', debug=0, debugger_theme='white', demo='', dense_wh=False, down_ratio=4, exp_id='default', fix_res=True, flip=0.5, flip_test=False, gpus=[0], gpus_str='0', head_conv=256, heads={'hm': 80, 'wh': 2, 'hm_rel': 117, 'sub_offset': 234, 'obj_offset': 234, 'reg': 2}, hide_data_time=False, hm_weight=1, image_dir='images/trainval', input_h=512, input_res=512, input_w=512, keep_res=False, load_model='', lr=0.000125, lr_step=[90, 120], master_batch_size=32, mean=array([[[0.40789655, 0.44719303, 0.47026116]]], dtype=float32), metric='loss', mse_loss=False, nms=False, no_color_aug=False, norm_wh=False, not_cuda_benchmark=False, not_prefetch_test=False, not_rand_crop=False, not_reg_offset=False, num_classes=80, num_classes_verb=117, num_epochs=140, num_iters=-1, num_stacks=1, num_workers=4, off_weight=1, output_h=1

In [5]:
model = create_model(opt.arch, opt.heads, opt.head_conv) 

In [6]:
print(model)

DLASeg(
  (base): DLA(
    (base_layer): Sequential(
      (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (level0): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (level1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (level2): Tree(
      (tree1): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inpla

In [61]:
inp = torch.rand(2,3,512,512)

In [11]:
model.cuda()

DLASeg(
  (base): DLA(
    (base_layer): Sequential(
      (0): Conv2d(3, 16, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (level0): Sequential(
      (0): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (level1): Sequential(
      (0): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): ReLU(inplace)
    )
    (level2): Tree(
      (tree1): BasicBlock(
        (conv1): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inpla

In [62]:
out = model(inp.cuda())

In [63]:
for key in out[0]:
    print(key,out[0][key].shape)

hm torch.Size([2, 80, 128, 128])
wh torch.Size([2, 2, 128, 128])
hm_rel torch.Size([2, 117, 128, 128])
sub_offset torch.Size([2, 234, 128, 128])
obj_offset torch.Size([2, 234, 128, 128])
reg torch.Size([2, 2, 128, 128])


In [18]:
import json
    

In [22]:
hoi_annotation = json.load(open('../Dataset/hico_det/annotations/trainval_hico.json','r'))

In [23]:
type(hoi_annotation)

list

In [38]:
hoi_annotation[9]

{'file_name': 'HICO_train2015_00000010.jpg',
 'img_id': 10,
 'annotations': [{'bbox': [266, 69, 369, 273], 'category_id': 1},
  {'bbox': [111, 69, 442, 410], 'category_id': 19},
  {'bbox': [282, 65, 353, 288], 'category_id': 1},
  {'bbox': [97, 68, 448, 416], 'category_id': 19}],
 'hoi_annotation': [{'subject_id': 0,
   'object_id': 1,
   'category_id': 37,
   'hoi_category_id': 132},
  {'subject_id': 0, 'object_id': 1, 'category_id': 77, 'hoi_category_id': 140},
  {'subject_id': 2,
   'object_id': 3,
   'category_id': 99,
   'hoi_category_id': 142}]}

In [25]:
len(hoi_annotation)

37633

In [17]:
inp.new_zeros((8,6))

tensor([[0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0.]])

In [18]:
inp

tensor([[[[6.4284e-01, 4.5300e-01, 1.6063e-01,  ..., 8.3155e-01,
           6.7112e-01, 7.4838e-02],
          [2.5540e-01, 5.5879e-01, 2.9102e-01,  ..., 1.9157e-02,
           7.3649e-01, 9.4765e-01],
          [9.7452e-01, 6.1641e-01, 2.4284e-01,  ..., 1.6031e-01,
           9.7933e-01, 1.6624e-01],
          ...,
          [3.0959e-02, 8.5629e-01, 4.1204e-01,  ..., 1.4710e-01,
           5.7472e-01, 9.9365e-01],
          [6.3456e-01, 2.4013e-01, 8.8683e-01,  ..., 9.6062e-01,
           2.2913e-01, 9.9980e-01],
          [1.8317e-01, 8.2605e-01, 8.7873e-01,  ..., 9.0550e-01,
           9.7295e-01, 8.4975e-01]],

         [[1.5791e-01, 7.9024e-01, 1.6684e-01,  ..., 3.7241e-01,
           6.7908e-01, 9.9644e-01],
          [6.6486e-01, 2.0115e-02, 9.6444e-01,  ..., 1.8604e-01,
           6.0808e-01, 9.3466e-01],
          [3.7497e-01, 7.2220e-01, 8.1125e-01,  ..., 6.7297e-01,
           5.8954e-01, 7.1415e-02],
          ...,
          [2.6642e-01, 3.2239e-01, 3.9868e-02,  ..., 4.4832

In [19]:
_valid_ids_verb = list(range(118))
_valid_ids_verb.remove(0)

In [21]:
cat_ids_verb = {v: i for i, v in enumerate(_valid_ids_verb)}

In [22]:
cat_ids_verb

{1: 0,
 2: 1,
 3: 2,
 4: 3,
 5: 4,
 6: 5,
 7: 6,
 8: 7,
 9: 8,
 10: 9,
 11: 10,
 12: 11,
 13: 12,
 14: 13,
 15: 14,
 16: 15,
 17: 16,
 18: 17,
 19: 18,
 20: 19,
 21: 20,
 22: 21,
 23: 22,
 24: 23,
 25: 24,
 26: 25,
 27: 26,
 28: 27,
 29: 28,
 30: 29,
 31: 30,
 32: 31,
 33: 32,
 34: 33,
 35: 34,
 36: 35,
 37: 36,
 38: 37,
 39: 38,
 40: 39,
 41: 40,
 42: 41,
 43: 42,
 44: 43,
 45: 44,
 46: 45,
 47: 46,
 48: 47,
 49: 48,
 50: 49,
 51: 50,
 52: 51,
 53: 52,
 54: 53,
 55: 54,
 56: 55,
 57: 56,
 58: 57,
 59: 58,
 60: 59,
 61: 60,
 62: 61,
 63: 62,
 64: 63,
 65: 64,
 66: 65,
 67: 66,
 68: 67,
 69: 68,
 70: 69,
 71: 70,
 72: 71,
 73: 72,
 74: 73,
 75: 74,
 76: 75,
 77: 76,
 78: 77,
 79: 78,
 80: 79,
 81: 80,
 82: 81,
 83: 82,
 84: 83,
 85: 84,
 86: 85,
 87: 86,
 88: 87,
 89: 88,
 90: 89,
 91: 90,
 92: 91,
 93: 92,
 94: 93,
 95: 94,
 96: 95,
 97: 96,
 98: 97,
 99: 98,
 100: 99,
 101: 100,
 102: 101,
 103: 102,
 104: 103,
 105: 104,
 106: 105,
 107: 106,
 108: 107,
 109: 108,
 110: 109,
 111: 11

In [23]:
def draw_gaussian_mask(heatmap, center, sigma):
  tmp_size = sigma * 3
  mu_x = int(center[0] + 0.5)
  mu_y = int(center[1] + 0.5)
  w, h = heatmap.shape[0], heatmap.shape[1]
  ul = [int(mu_x - tmp_size), int(mu_y - tmp_size)]
  br = [int(mu_x + tmp_size + 1), int(mu_y + tmp_size + 1)]
  if ul[0] >= h or ul[1] >= w or br[0] < 0 or br[1] < 0:
    return heatmap
  size = 2 * tmp_size + 1
  x = np.arange(0, size, 1, np.float32)
  y = x[:, np.newaxis]
  x0 = y0 = size // 2
  g = np.exp(- ((x - x0) ** 2 + (y - y0) ** 2) / (2 * sigma ** 2))
  g_x = max(0, -ul[0]), min(br[0], h) - ul[0]
  g_y = max(0, -ul[1]), min(br[1], w) - ul[1]
  img_x = max(0, ul[0]), min(br[0], h)
  img_y = max(0, ul[1]), min(br[1], w)
  heatmap[img_y[0]:img_y[1], img_x[0]:img_x[1]] = np.maximum(
    heatmap[img_y[0]:img_y[1], img_x[0]:img_x[1]],
    g[g_y[0]:g_y[1], g_x[0]:g_x[1]])
  
  return heatmap

In [25]:
import numpy as np

In [30]:
heatmap = np.zeros((8,8),dtype=np.float32)

In [31]:
x = draw_gaussian_mask(heatmap,(4,4),1)

In [34]:
x[x>0]=1

In [35]:
x

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)

In [36]:
heatmap

array([[0., 0., 0., 0., 0., 0., 0., 0.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.],
       [0., 1., 1., 1., 1., 1., 1., 1.]], dtype=float32)

In [37]:
point = [0,1]

In [38]:
x,y = point

In [39]:
x

0

In [55]:
offset = np.zeros((8, 2, 8, 8), dtype=np.float32)

In [56]:
offset.shape

(8, 2, 8, 8)

In [57]:
off = offset[0]

In [47]:
off.shape[0]

2

In [48]:
for i in range(off.shape[1]):
    for j in range(off.shape[2]):
        off[:,i,j] = i-0,j-1

In [58]:
off

array([[[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0.]]], dtype=float32)

In [53]:
off[:,1,3]

array([1., 2.], dtype=float32)

In [54]:
def draw_offset(offset_heatmap, point):
  x,y = point
  for i in range(offset_heatmap.shape[1]):
      for j in range(offset_heatmap.shape[2]):
          offset_heatmap[:,i,j] = i-x,j-y
  return offset_heatmap  

In [59]:
draw_offset(off,[0,1])

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
        [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.]],

       [[-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.]]], dtype=float32)

In [60]:
off

array([[[ 0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.],
        [ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.],
        [ 2.,  2.,  2.,  2.,  2.,  2.,  2.,  2.],
        [ 3.,  3.,  3.,  3.,  3.,  3.,  3.,  3.],
        [ 4.,  4.,  4.,  4.,  4.,  4.,  4.,  4.],
        [ 5.,  5.,  5.,  5.,  5.,  5.,  5.,  5.],
        [ 6.,  6.,  6.,  6.,  6.,  6.,  6.,  6.],
        [ 7.,  7.,  7.,  7.,  7.,  7.,  7.,  7.]],

       [[-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.],
        [-1.,  0.,  1.,  2.,  3.,  4.,  5.,  6.]]], dtype=float32)

In [65]:
def _gather_feat(feat, ind, mask=None):
    dim  = feat.size(2)
    ind  = ind.unsqueeze(2).expand(ind.size(0), ind.size(1), dim)
    feat = feat.gather(1, ind)
    if mask is not None:
        mask = mask.unsqueeze(2).expand_as(feat)
        feat = feat[mask]
        feat = feat.view(-1, dim)
    return feat

In [64]:
 def _tranpose_and_gather_feat(feat, ind):
    feat = feat.permute(0, 2, 3, 1).contiguous()
    feat = feat.view(feat.size(0), -1, feat.size(3))
    feat = _gather_feat(feat, ind)
    return feat

In [93]:
feat = torch.rand(2,2,128,128)

In [115]:
ind = torch.randint(128*128,(2,10))

In [116]:
ind

tensor([[ 2008., 13462.,  2232.,  3353.,   143., 14115.,  3965.,  1834.,  1802.,
          5814.],
        [ 8047.,  6783., 12222.,  1929., 11991., 15780., 11963.,  5402.,  3527.,
         10543.]])

In [117]:
ind  = ind.unsqueeze(2).expand(ind.size(0), ind.size(1), 2)

In [119]:
ind

tensor([[[ 2008.,  2008.],
         [13462., 13462.],
         [ 2232.,  2232.],
         [ 3353.,  3353.],
         [  143.,   143.],
         [14115., 14115.],
         [ 3965.,  3965.],
         [ 1834.,  1834.],
         [ 1802.,  1802.],
         [ 5814.,  5814.]],

        [[ 8047.,  8047.],
         [ 6783.,  6783.],
         [12222., 12222.],
         [ 1929.,  1929.],
         [11991., 11991.],
         [15780., 15780.],
         [11963., 11963.],
         [ 5402.,  5402.],
         [ 3527.,  3527.],
         [10543., 10543.]]])

In [77]:
feat = feat.permute(0, 2, 3, 1).contiguous()

In [79]:
feat = feat.view(feat.size(0), -1, feat.size(3))

In [80]:
feat.shape

torch.Size([2, 16384, 234])

In [81]:
dim  = feat.size(2)
ind  = ind.unsqueeze(2).expand(ind.size(0), ind.size(1), dim)

In [95]:
ind = ind.long()

In [96]:
x = _tranpose_and_gather_feat(feat, ind)

In [97]:
x.shape

torch.Size([2, 10, 2])

In [101]:
mask = torch.rand(2,10)

In [104]:
mask[mask>0] = 1

In [106]:
mask = mask.unsqueeze(2).expand_as(x).float()

In [113]:
mask.shape

torch.Size([2, 10, 2])

In [109]:
suboffset = torch.rand(2,10,2,128,128)

In [114]:
suboffset.shape

torch.Size([2, 10, 2, 128, 128])

In [112]:
suboffset*mask

RuntimeError: The size of tensor a (128) must match the size of tensor b (2) at non-singleton dimension 4

In [120]:
import torch.nn.functional as F

In [121]:
pred = torch.rand(2,234,128,128)
target = torch.rand(2,234,128,128)

In [122]:
loss = F.l1_loss(pred, target)

In [123]:
loss

tensor(0.3316)

In [166]:
output = torch.rand(2,234,4,4)
id = torch.randint(117,(2,4)).long()

In [142]:
output.shape

torch.Size([2, 234, 4, 4])

In [173]:
x= output.view(output.size(0),-1,2,output.size(2),output.size(3))

In [174]:
x.shape

torch.Size([2, 117, 2, 4, 4])

In [167]:
id.shape

torch.Size([2, 4])

In [168]:
id

tensor([[105,   1,  91,  65],
        [111,  89,  37,   8]])

In [169]:
id = id.unsqueeze(2).unsqueeze(3).unsqueeze(4).expand(id.size(0), id.size(1), 2, 4, 4)

In [170]:
id.shape

torch.Size([2, 4, 2, 4, 4])

In [175]:
x.shape

torch.Size([2, 117, 2, 4, 4])

In [185]:
x[1,111,...]

tensor([[[0.2596, 0.8252, 0.1158, 0.2447],
         [0.7099, 0.2261, 0.2139, 0.8317],
         [0.9090, 0.4839, 0.4155, 0.3707],
         [0.9032, 0.8130, 0.8953, 0.6375]],

        [[0.3199, 0.1705, 0.3966, 0.7048],
         [0.5049, 0.6957, 0.9793, 0.9772],
         [0.9594, 0.4095, 0.5173, 0.2640],
         [0.4259, 0.7042, 0.3473, 0.9461]]])

In [203]:
y = x.gather(1,id)

In [211]:
y.shape

torch.Size([2, 4, 2, 4, 4])

In [205]:
y[0][0]

tensor([[[0.9467, 0.2668, 0.2382, 0.3521],
         [0.2258, 0.0303, 0.7459, 0.9404],
         [0.3269, 0.3035, 0.3804, 0.2106],
         [0.9162, 0.1702, 0.7730, 0.5266]],

        [[0.6122, 0.1753, 0.8452, 0.6629],
         [0.2326, 0.7061, 0.5225, 0.4258],
         [0.4373, 0.4352, 0.3213, 0.9018],
         [0.6343, 0.4178, 0.2191, 0.2581]]])

In [212]:
mask = torch.rand(2,4,4,4)

In [213]:
mask

tensor([[[[0.4973, 0.0332, 0.2315, 0.8644],
          [0.6310, 0.3141, 0.2344, 0.8679],
          [0.3102, 0.7997, 0.8978, 0.1184],
          [0.0268, 0.7056, 0.1279, 0.1426]],

         [[0.2721, 0.1636, 0.9796, 0.6561],
          [0.7841, 0.0904, 0.2888, 0.7617],
          [0.2891, 0.6875, 0.3316, 0.2118],
          [0.0512, 0.4419, 0.1532, 0.2235]],

         [[0.0761, 0.1278, 0.8982, 0.0845],
          [0.9963, 0.6068, 0.2757, 0.7953],
          [0.0710, 0.7914, 0.9398, 0.9089],
          [0.9368, 0.2734, 0.2056, 0.9034]],

         [[0.5141, 0.6622, 0.2205, 0.7425],
          [0.1222, 0.5994, 0.1914, 0.5262],
          [0.0427, 0.5635, 0.6132, 0.8983],
          [0.7985, 0.5426, 0.7088, 0.9586]]],


        [[[0.7212, 0.6055, 0.0393, 0.4090],
          [0.9266, 0.6836, 0.6747, 0.7840],
          [0.1797, 0.5415, 0.3222, 0.4587],
          [0.7273, 0.4992, 0.6190, 0.5999]],

         [[0.2046, 0.4445, 0.6859, 0.2021],
          [0.2625, 0.3422, 0.1228, 0.8654],
          [0.3987, 0

In [215]:
mask = mask.unsqueeze(2).expand_as(y)

In [216]:
mask*y

tensor([[[[[0.4708, 0.0088, 0.0551, 0.3043],
           [0.1425, 0.0095, 0.1748, 0.8162],
           [0.1014, 0.2427, 0.3415, 0.0249],
           [0.0246, 0.1201, 0.0988, 0.0751]],

          [[0.3044, 0.0058, 0.1956, 0.5730],
           [0.1468, 0.2218, 0.1225, 0.3695],
           [0.1357, 0.3481, 0.2885, 0.1068],
           [0.0170, 0.2948, 0.0280, 0.0368]]],


         [[[0.0043, 0.0890, 0.9479, 0.3155],
           [0.2385, 0.0095, 0.0611, 0.5544],
           [0.2785, 0.2126, 0.2259, 0.1132],
           [0.0246, 0.3990, 0.0978, 0.1498]],

          [[0.1547, 0.1405, 0.8287, 0.4969],
           [0.7757, 0.0391, 0.2874, 0.4244],
           [0.2253, 0.0408, 0.0226, 0.0851],
           [0.0036, 0.0173, 0.0999, 0.0546]]],


         [[[0.0740, 0.0478, 0.1905, 0.0162],
           [0.3143, 0.5649, 0.2359, 0.6844],
           [0.0619, 0.3187, 0.0799, 0.4072],
           [0.3117, 0.2551, 0.1199, 0.8399]],

          [[0.0033, 0.0528, 0.6521, 0.0039],
           [0.3385, 0.2255, 0.0679, 0.394

In [202]:
y.view(y.size(0),-1,y.size(3),y.size(4)).shape

torch.Size([2, 8, 4, 4])

In [137]:
output.gather(1,id)

RuntimeError: invalid argument 4: Index tensor must have same dimensions as input tensor at /opt/conda/conda-bld/pytorch_1535493744281/work/aten/src/TH/generic/THTensorMath.cpp:601

In [145]:
x = torch.rand(2,6,2,2)

In [148]:
y = x.view(x.size(0),-1,2,x.size(2),x.size(3))

In [154]:
x[0,0:3,:,:]

tensor([[[0.9014, 0.7832],
         [0.0354, 0.2964]],

        [[0.4467, 0.0210],
         [0.1566, 0.4616]],

        [[0.3397, 0.6119],
         [0.8065, 0.9524]]])

In [155]:
y[0,:,1,:,:]

tensor([[[0.4467, 0.0210],
         [0.1566, 0.4616]],

        [[0.8846, 0.4512],
         [0.9043, 0.6589]],

        [[0.6576, 0.8447],
         [0.3778, 0.5791]]])

In [218]:
import numpy as np
a = np.array([0,0])
a/0

/home/xh/miniconda3/envs/pytorch/lib/python3.7/site-packages/ipykernel_launcher.py:3: RuntimeWarning: invalid value encountered in true_divide
  This is separate from the ipykernel package so we can avoid doing imports until


array([nan, nan])

In [221]:
class OffsetLoss(nn.Module):
  def __init__(self):
    super(OffsetLoss, self).__init__()
  
  def forward(self, output, mask, ind, target, rel_id, heatmap_mask):
    pred = _offset_format(output, rel_id)
    heatmap_mask = heatmap_mask.unsqueeze(2).expand_as(pred).float()
    # loss = F.l1_loss(pred * heatmap_mask, target * heatmap_mask, reduction='elementwise_mean')
    loss = F.l1_loss(pred * heatmap_mask, target * heatmap_mask, size_average=False)
    mask = mask.float()
    loss = loss / (mask.sum() + 1e-4)
    return loss

In [220]:
import torch.nn as nn

In [222]:
loss = OffsetLoss()